In [1]:
for name in list(globals()):
    if not name.startswith('_'):
        del globals()[name]


import os
import pandas as pd
import numpy as np

###############################################################################################################
################START OF MAIN CODE###############################################################################
###############################################################################################################


# This code gets the implied volatility surface using the estimated SVI coefficients.
# Some other stuff can be done when the cluster dates are used, but this is not of main interest in this code
# and I do not remember what it was used for.

# Path to read in the IV Matrix files for each day. Use the IR0 folder in the github.
path="E:/share/SVI"
#path="/Users/ratmir/Downloads/testttm/IV_matrix_IR0"

os.chdir(path)

#This is the input from the SVI estimation, that we got from estimate_SVI_coefficients_each_day.py:
df1 = pd.read_csv('svi_iv_and_r2_results14b.csv')
df2 = pd.read_csv('paras14b.csv')

#C1 = pd.read_csv('dates_cluster1.csv')
#C0 = pd.read_csv('dates_cluster0.csv')


#these .pngs should be deleted
# del_dates = df1[df1['R2'] < 0.97].iloc[:, 0]
# folder_path = '/Users/ratmir/Desktop/Plots_IVs'  # Replace with the path to your folder containing the .png files
# for entry in del_dates:
#     # Extract the base name from the entry string (i.e., without .csv) and construct the .png filename
#     base_name = entry.replace('.csv', '')
#     png_filename = base_name + '_plot.png'
#
#     # Create the full path to the .png file
#     full_path = os.path.join(folder_path, png_filename)
#
#     # Check if the file exists and then delete it
#     if os.path.exists(full_path):
#         os.remove(full_path)
#         print(f"Deleted {png_filename}")
#     else:
#         print(f"{png_filename} not found")

R2_threshold = 0.98
df1 = df1[df1['R2'] >= R2_threshold] # 0.97; 761 remain # 0.98; 21 remain
paras = df2.loc[df1.index]
paras.columns.values[0] = 'Date'

print(paras.columns)
paras.columns = paras.columns.str.strip()

def svi_model(theta, k, tau):
    base_params = np.array(theta[:5])
    ttm_coeffs = np.array(theta[5:])
    a, b, rho, m, sigma = base_params + ttm_coeffs * tau
    return a + b * (rho * (k - m) + np.sqrt((k - m) ** 2 + sigma ** 2))


#You have to change the range of k_new, if you want to have a different range. Maybe this is what you want Zijin:
k_new = np.linspace(-1, 1, 2001)
all_IVs = pd.DataFrame()
all_IVs_list = []
#ttms = list(range(3, 41))
# Set to other ttms, if you want to have the IVs for other ttms.
ttms = list([5, 9, 14, 27])  # or 9 9 , or 14 14 or 27 27

for ttm in ttms:
    print(f"Interpolate IV for ttm {ttm}.")
    all_IVs = pd.DataFrame()  # Initialize an empty DataFrame for the current ttm

    for file in paras['Date']:

        #extracting thetas
        theta = paras.loc[paras['Date'] == file]
        theta = theta.drop(theta.columns[0], axis=1)
        theta = np.squeeze(theta.values)

        IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)

        # Assuming IV is a list or array, convert IV into a dictionary with column names
        IV_dict = {str(k): [iv] for k, iv in zip(k_new, IV)}
        IV_dict['Date'] = file

        # Convert dictionary to DataFrame
        IV_df = pd.DataFrame(IV_dict)

        # Check for NaN values
        if IV_df.isna().any().any():
            print(f"NaN values found in IV for {file}.")
            continue

        # Ensure 'Date' is the first column
        cols = ['Date'] + [col for col in IV_df if col != 'Date']
        IV_df = IV_df[cols]

        # Append to the main DataFrame
        all_IVs = pd.concat([all_IVs, IV_df], ignore_index=True)

    print(all_IVs['Date'].str)
    all_IVs['Date'] = all_IVs['Date'].str.extract(r'(\d{4}-\d{2}-\d{2})')
    all_IVs.to_csv(f'IR0/moneyness/interpolated_IV{ttm}_098.csv', index=False)

    all_IVs_list.append(all_IVs)




Index(['Date', 'a', 'b', 'rho', 'm', 'sigma', 'a_ttm', 'b_ttm', 'rho_ttm',
       'm_ttm', 'sigma_ttm'],
      dtype='object')
Interpolate IV for ttm 5.
NaN values found in IV for IV_matrix_2017-11-25.csv.
NaN values found in IV for IV_matrix_2018-03-09.csv.


C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\312718561.py:85: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)
C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\312718561.py:85: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)


Interpolate IV for ttm 9.
Interpolate IV for ttm 14.
Interpolate IV for ttm 27.
NaN values found in IV for IV_matrix_2022-11-20.csv.


C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\312718561.py:85: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)


In [2]:
R2_threshold = 0.95

df1 = pd.read_csv('svi_iv_and_r2_results14b.csv')
df2 = pd.read_csv('paras14b.csv')
df1 = df1[df1['R2'] >= R2_threshold] # 0.97; 761 remain # 0.98; 21 remain
paras = df2.loc[df1.index]
paras.columns.values[0] = 'Date'

print(paras.columns)
paras.columns = paras.columns.str.strip()

def svi_model(theta, k, tau):
    base_params = np.array(theta[:5])
    ttm_coeffs = np.array(theta[5:])
    a, b, rho, m, sigma = base_params + ttm_coeffs * tau
    return a + b * (rho * (k - m) + np.sqrt((k - m) ** 2 + sigma ** 2))


#You have to change the range of k_new, if you want to have a different range. Maybe this is what you want Zijin:
k_new = np.linspace(-1, 1, 2001)
all_IVs = pd.DataFrame()
all_IVs_list = []
#ttms = list(range(3, 41))
# Set to other ttms, if you want to have the IVs for other ttms.
ttms = list([5, 9, 14, 27])  # or 9 9 , or 14 14 or 27 27

for ttm in ttms:
    print(f"Interpolate IV for ttm {ttm}.")
    all_IVs = pd.DataFrame()  # Initialize an empty DataFrame for the current ttm

    for file in paras['Date']:

        #extracting thetas
        theta = paras.loc[paras['Date'] == file]
        theta = theta.drop(theta.columns[0], axis=1)
        theta = np.squeeze(theta.values)

        IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)

        # Assuming IV is a list or array, convert IV into a dictionary with column names
        IV_dict = {str(k): [iv] for k, iv in zip(k_new, IV)}
        IV_dict['Date'] = file

        # Convert dictionary to DataFrame
        IV_df = pd.DataFrame(IV_dict)

        # Check for NaN values
        if IV_df.isna().any().any():
            print(f"NaN values found in IV for {file}.")
            continue

        # Ensure 'Date' is the first column
        cols = ['Date'] + [col for col in IV_df if col != 'Date']
        IV_df = IV_df[cols]

        # Append to the main DataFrame
        all_IVs = pd.concat([all_IVs, IV_df], ignore_index=True)

    print(all_IVs['Date'].str)
    all_IVs['Date'] = all_IVs['Date'].str.extract(r'(\d{4}-\d{2}-\d{2})')
    all_IVs.to_csv(f'IR0/moneyness/interpolated_IV{ttm}_095.csv', index=False)

    all_IVs_list.append(all_IVs)

Index(['Date', 'a', 'b', 'rho', 'm', 'sigma', 'a_ttm', 'b_ttm', 'rho_ttm',
       'm_ttm', 'sigma_ttm'],
      dtype='object')
Interpolate IV for ttm 5.
NaN values found in IV for IV_matrix_2017-11-25.csv.


C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\1711478381.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)
C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\1711478381.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)
C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\1711478381.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)


NaN values found in IV for IV_matrix_2018-03-09.csv.
NaN values found in IV for IV_matrix_2018-05-24.csv.
NaN values found in IV for IV_matrix_2018-05-25.csv.
NaN values found in IV for IV_matrix_2018-06-08.csv.
NaN values found in IV for IV_matrix_2018-07-25.csv.
NaN values found in IV for IV_matrix_2018-08-01.csv.


C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\1711478381.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)
C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\1711478381.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)
C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\1711478381.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)
C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\1711478381.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)
C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\1711478381.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)


NaN values found in IV for IV_matrix_2018-09-26.csv.
NaN values found in IV for IV_matrix_2019-02-13.csv.


...

NaN values found in IV for IV_matrix_2018-07-25.csv.
NaN values found in IV for IV_matrix_2018-08-01.csv.
NaN values found in IV for IV_matrix_2018-09-26.csv.
Interpolate IV for ttm 14.


C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\1711478381.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)
C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\1711478381.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)
C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\1711478381.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)


NaN values found in IV for IV_matrix_2018-07-25.csv.
NaN values found in IV for IV_matrix_2018-08-01.csv.
NaN values found in IV for IV_matrix_2018-09-26.csv.
Interpolate IV for ttm 27.


C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\1711478381.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)
C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\1711478381.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)


NaN values found in IV for IV_matrix_2022-11-20.csv.
NaN values found in IV for IV_matrix_2022-11-28.csv.


In [3]:
R2_threshold = 0.9

df1 = pd.read_csv('svi_iv_and_r2_results14b.csv')
df2 = pd.read_csv('paras14b.csv')
df1 = df1[df1['R2'] >= R2_threshold] # 0.97; 761 remain # 0.98; 21 remain
paras = df2.loc[df1.index]
paras.columns.values[0] = 'Date'

print(paras.columns)
paras.columns = paras.columns.str.strip()

def svi_model(theta, k, tau):
    base_params = np.array(theta[:5])
    ttm_coeffs = np.array(theta[5:])
    a, b, rho, m, sigma = base_params + ttm_coeffs * tau
    return a + b * (rho * (k - m) + np.sqrt((k - m) ** 2 + sigma ** 2))


#You have to change the range of k_new, if you want to have a different range. Maybe this is what you want Zijin:
k_new = np.linspace(-1, 1, 2001)
all_IVs = pd.DataFrame()
all_IVs_list = []
#ttms = list(range(3, 41))
# Set to other ttms, if you want to have the IVs for other ttms.
ttms = list([5, 9, 14, 27])  # or 9 9 , or 14 14 or 27 27

for ttm in ttms:
    print(f"Interpolate IV for ttm {ttm}.")
    all_IVs = pd.DataFrame()  # Initialize an empty DataFrame for the current ttm

    for file in paras['Date']:

        #extracting thetas
        theta = paras.loc[paras['Date'] == file]
        theta = theta.drop(theta.columns[0], axis=1)
        theta = np.squeeze(theta.values)

        IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)

        # Assuming IV is a list or array, convert IV into a dictionary with column names
        IV_dict = {str(k): [iv] for k, iv in zip(k_new, IV)}
        IV_dict['Date'] = file

        # Convert dictionary to DataFrame
        IV_df = pd.DataFrame(IV_dict)

        # Check for NaN values
        if IV_df.isna().any().any():
            print(f"NaN values found in IV for {file}.")
            continue

        # Ensure 'Date' is the first column
        cols = ['Date'] + [col for col in IV_df if col != 'Date']
        IV_df = IV_df[cols]

        # Append to the main DataFrame
        all_IVs = pd.concat([all_IVs, IV_df], ignore_index=True)

    print(all_IVs['Date'].str)
    all_IVs['Date'] = all_IVs['Date'].str.extract(r'(\d{4}-\d{2}-\d{2})')
    all_IVs.to_csv(f'IR0/moneyness/interpolated_IV{ttm}_090.csv', index=False)

    all_IVs_list.append(all_IVs)


Index(['Date', 'a', 'b', 'rho', 'm', 'sigma', 'a_ttm', 'b_ttm', 'rho_ttm',
       'm_ttm', 'sigma_ttm'],
      dtype='object')
Interpolate IV for ttm 5.
NaN values found in IV for IV_matrix_2017-08-14.csv.


C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\832053914.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)
C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\832053914.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)


NaN values found in IV for IV_matrix_2017-11-25.csv.
NaN values found in IV for IV_matrix_2017-12-02.csv.


C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\832053914.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)
C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\832053914.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)


NaN values found in IV for IV_matrix_2018-03-09.csv.


...

Interpolate IV for ttm 27.


C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\832053914.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)
C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\832053914.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)
C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\832053914.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)
C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\832053914.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)


NaN values found in IV for IV_matrix_2022-11-10.csv.
NaN values found in IV for IV_matrix_2022-11-13.csv.
NaN values found in IV for IV_matrix_2022-11-14.csv.
NaN values found in IV for IV_matrix_2022-11-15.csv.


C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\832053914.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)
C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\832053914.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)
C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\832053914.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)


NaN values found in IV for IV_matrix_2022-11-20.csv.
NaN values found in IV for IV_matrix_2022-11-22.csv.
NaN values found in IV for IV_matrix_2022-11-28.csv.


In [4]:
R2_threshold = 0.8

df1 = pd.read_csv('svi_iv_and_r2_results14b.csv')
df2 = pd.read_csv('paras14b.csv')
df1 = df1[df1['R2'] >= R2_threshold] # 0.97; 761 remain # 0.98; 21 remain
paras = df2.loc[df1.index]
paras.columns.values[0] = 'Date'

print(paras.columns)
paras.columns = paras.columns.str.strip()

def svi_model(theta, k, tau):
    base_params = np.array(theta[:5])
    ttm_coeffs = np.array(theta[5:])
    a, b, rho, m, sigma = base_params + ttm_coeffs * tau
    return a + b * (rho * (k - m) + np.sqrt((k - m) ** 2 + sigma ** 2))


#You have to change the range of k_new, if you want to have a different range. Maybe this is what you want Zijin:
k_new = np.linspace(-1, 1, 2001)
all_IVs = pd.DataFrame()
all_IVs_list = []
#ttms = list(range(3, 41))
# Set to other ttms, if you want to have the IVs for other ttms.
ttms = list([5, 9, 14, 27])  # or 9 9 , or 14 14 or 27 27

for ttm in ttms:
    print(f"Interpolate IV for ttm {ttm}.")
    all_IVs = pd.DataFrame()  # Initialize an empty DataFrame for the current ttm

    for file in paras['Date']:

        #extracting thetas
        theta = paras.loc[paras['Date'] == file]
        theta = theta.drop(theta.columns[0], axis=1)
        theta = np.squeeze(theta.values)

        IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)

        # Assuming IV is a list or array, convert IV into a dictionary with column names
        IV_dict = {str(k): [iv] for k, iv in zip(k_new, IV)}
        IV_dict['Date'] = file

        # Convert dictionary to DataFrame
        IV_df = pd.DataFrame(IV_dict)

        # Check for NaN values
        if IV_df.isna().any().any():
            print(f"NaN values found in IV for {file}.")
            continue

        # Ensure 'Date' is the first column
        cols = ['Date'] + [col for col in IV_df if col != 'Date']
        IV_df = IV_df[cols]

        # Append to the main DataFrame
        all_IVs = pd.concat([all_IVs, IV_df], ignore_index=True)

    print(all_IVs['Date'].str)
    all_IVs['Date'] = all_IVs['Date'].str.extract(r'(\d{4}-\d{2}-\d{2})')
    all_IVs.to_csv(f'IR0/moneyness/interpolated_IV{ttm}_080.csv', index=False)

    all_IVs_list.append(all_IVs)

Index(['Date', 'a', 'b', 'rho', 'm', 'sigma', 'a_ttm', 'b_ttm', 'rho_ttm',
       'm_ttm', 'sigma_ttm'],
      dtype='object')
Interpolate IV for ttm 5.
NaN values found in IV for IV_matrix_2017-08-14.csv.


C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\840179055.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)
C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\840179055.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)
C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\840179055.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)
C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\840179055.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)


NaN values found in IV for IV_matrix_2017-11-25.csv.
NaN values found in IV for IV_matrix_2017-12-02.csv.
NaN values found in IV for IV_matrix_2017-12-08.csv.


C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\840179055.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)


NaN values found in IV for IV_matrix_2017-12-15.csv.


...

NaN values found in IV for IV_matrix_2018-09-26.csv.
NaN values found in IV for IV_matrix_2018-10-03.csv.
NaN values found in IV for IV_matrix_2018-10-31.csv.
NaN values found in IV for IV_matrix_2018-11-07.csv.
Interpolate IV for ttm 27.


C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\840179055.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)
C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\840179055.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)
C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\840179055.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)
C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\840179055.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)


NaN values found in IV for IV_matrix_2022-11-10.csv.
NaN values found in IV for IV_matrix_2022-11-13.csv.
NaN values found in IV for IV_matrix_2022-11-14.csv.
NaN values found in IV for IV_matrix_2022-11-15.csv.


C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\840179055.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)
C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\840179055.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)
C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\840179055.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)


NaN values found in IV for IV_matrix_2022-11-20.csv.
NaN values found in IV for IV_matrix_2022-11-22.csv.
NaN values found in IV for IV_matrix_2022-11-28.csv.


In [5]:
R2_threshold = 0.7

df1 = pd.read_csv('svi_iv_and_r2_results14b.csv')
df2 = pd.read_csv('paras14b.csv')
df1 = df1[df1['R2'] >= R2_threshold] # 0.97; 761 remain # 0.98; 21 remain
paras = df2.loc[df1.index]
paras.columns.values[0] = 'Date'

print(paras.columns)
paras.columns = paras.columns.str.strip()

def svi_model(theta, k, tau):
    base_params = np.array(theta[:5])
    ttm_coeffs = np.array(theta[5:])
    a, b, rho, m, sigma = base_params + ttm_coeffs * tau
    return a + b * (rho * (k - m) + np.sqrt((k - m) ** 2 + sigma ** 2))


#You have to change the range of k_new, if you want to have a different range. Maybe this is what you want Zijin:
k_new = np.linspace(-1, 1, 2001)
all_IVs = pd.DataFrame()
all_IVs_list = []
#ttms = list(range(3, 41))
# Set to other ttms, if you want to have the IVs for other ttms.
ttms = list([5, 9, 14, 27])  # or 9 9 , or 14 14 or 27 27

for ttm in ttms:
    print(f"Interpolate IV for ttm {ttm}.")
    all_IVs = pd.DataFrame()  # Initialize an empty DataFrame for the current ttm

    for file in paras['Date']:

        #extracting thetas
        theta = paras.loc[paras['Date'] == file]
        theta = theta.drop(theta.columns[0], axis=1)
        theta = np.squeeze(theta.values)

        IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)

        # Assuming IV is a list or array, convert IV into a dictionary with column names
        IV_dict = {str(k): [iv] for k, iv in zip(k_new, IV)}
        IV_dict['Date'] = file

        # Convert dictionary to DataFrame
        IV_df = pd.DataFrame(IV_dict)

        # Check for NaN values
        if IV_df.isna().any().any():
            print(f"NaN values found in IV for {file}.")
            continue

        # Ensure 'Date' is the first column
        cols = ['Date'] + [col for col in IV_df if col != 'Date']
        IV_df = IV_df[cols]

        # Append to the main DataFrame
        all_IVs = pd.concat([all_IVs, IV_df], ignore_index=True)

    print(all_IVs['Date'].str)
    all_IVs['Date'] = all_IVs['Date'].str.extract(r'(\d{4}-\d{2}-\d{2})')
    all_IVs.to_csv(f'IR0/moneyness/interpolated_IV{ttm}_070.csv', index=False)

    all_IVs_list.append(all_IVs)

Index(['Date', 'a', 'b', 'rho', 'm', 'sigma', 'a_ttm', 'b_ttm', 'rho_ttm',
       'm_ttm', 'sigma_ttm'],
      dtype='object')
Interpolate IV for ttm 5.
NaN values found in IV for IV_matrix_2017-08-14.csv.


C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\1215995090.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)
C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\1215995090.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)


NaN values found in IV for IV_matrix_2017-09-08.csv.


C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\1215995090.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)
C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\1215995090.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)
C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\1215995090.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)


NaN values found in IV for IV_matrix_2017-11-25.csv.
NaN values found in IV for IV_matrix_2017-12-02.csv.
NaN values found in IV for IV_matrix_2017-12-08.csv.


...

NaN values found in IV for IV_matrix_2019-01-23.csv.
Interpolate IV for ttm 27.


C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\1215995090.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)
C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\1215995090.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)
C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\1215995090.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)
C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\1215995090.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)


NaN values found in IV for IV_matrix_2022-11-10.csv.
NaN values found in IV for IV_matrix_2022-11-13.csv.
NaN values found in IV for IV_matrix_2022-11-14.csv.
NaN values found in IV for IV_matrix_2022-11-15.csv.


C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\1215995090.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)
C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\1215995090.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)
C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\1215995090.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)


NaN values found in IV for IV_matrix_2022-11-20.csv.
NaN values found in IV for IV_matrix_2022-11-22.csv.
NaN values found in IV for IV_matrix_2022-11-28.csv.


In [6]:
R2_threshold = 0.6

df1 = pd.read_csv('svi_iv_and_r2_results14b.csv')
df2 = pd.read_csv('paras14b.csv')
df1 = df1[df1['R2'] >= R2_threshold] # 0.97; 761 remain # 0.98; 21 remain
paras = df2.loc[df1.index]
paras.columns.values[0] = 'Date'

print(paras.columns)
paras.columns = paras.columns.str.strip()

def svi_model(theta, k, tau):
    base_params = np.array(theta[:5])
    ttm_coeffs = np.array(theta[5:])
    a, b, rho, m, sigma = base_params + ttm_coeffs * tau
    return a + b * (rho * (k - m) + np.sqrt((k - m) ** 2 + sigma ** 2))


#You have to change the range of k_new, if you want to have a different range. Maybe this is what you want Zijin:
k_new = np.linspace(-1, 1, 2001)
all_IVs = pd.DataFrame()
all_IVs_list = []
#ttms = list(range(3, 41))
# Set to other ttms, if you want to have the IVs for other ttms.
ttms = list([5, 9, 14, 27])  # or 9 9 , or 14 14 or 27 27

for ttm in ttms:
    print(f"Interpolate IV for ttm {ttm}.")
    all_IVs = pd.DataFrame()  # Initialize an empty DataFrame for the current ttm

    for file in paras['Date']:

        #extracting thetas
        theta = paras.loc[paras['Date'] == file]
        theta = theta.drop(theta.columns[0], axis=1)
        theta = np.squeeze(theta.values)

        IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)

        # Assuming IV is a list or array, convert IV into a dictionary with column names
        IV_dict = {str(k): [iv] for k, iv in zip(k_new, IV)}
        IV_dict['Date'] = file

        # Convert dictionary to DataFrame
        IV_df = pd.DataFrame(IV_dict)

        # Check for NaN values
        if IV_df.isna().any().any():
            print(f"NaN values found in IV for {file}.")
            continue

        # Ensure 'Date' is the first column
        cols = ['Date'] + [col for col in IV_df if col != 'Date']
        IV_df = IV_df[cols]

        # Append to the main DataFrame
        all_IVs = pd.concat([all_IVs, IV_df], ignore_index=True)

    print(all_IVs['Date'].str)
    all_IVs['Date'] = all_IVs['Date'].str.extract(r'(\d{4}-\d{2}-\d{2})')
    all_IVs.to_csv(f'IR0/moneyness/interpolated_IV{ttm}_060.csv', index=False)

    all_IVs_list.append(all_IVs)

Index(['Date', 'a', 'b', 'rho', 'm', 'sigma', 'a_ttm', 'b_ttm', 'rho_ttm',
       'm_ttm', 'sigma_ttm'],
      dtype='object')
Interpolate IV for ttm 5.
NaN values found in IV for IV_matrix_2017-08-14.csv.


C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\3482127528.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)
C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\3482127528.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)


NaN values found in IV for IV_matrix_2017-09-08.csv.


C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\3482127528.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)
C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\3482127528.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)
C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\3482127528.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)


NaN values found in IV for IV_matrix_2017-11-25.csv.
NaN values found in IV for IV_matrix_2017-12-02.csv.
NaN values found in IV for IV_matrix_2017-12-08.csv.
NaN values found in IV for IV_matrix_2017-12-15.csv.


...

NaN values found in IV for IV_matrix_2019-01-23.csv.
Interpolate IV for ttm 27.


C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\3482127528.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)
C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\3482127528.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)
C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\3482127528.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)
C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\3482127528.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)


NaN values found in IV for IV_matrix_2022-11-10.csv.
NaN values found in IV for IV_matrix_2022-11-13.csv.
NaN values found in IV for IV_matrix_2022-11-14.csv.
NaN values found in IV for IV_matrix_2022-11-15.csv.


C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\3482127528.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)
C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\3482127528.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)
C:\Users\Guti\AppData\Local\Temp\ipykernel_42152\3482127528.py:38: RuntimeWarning: invalid value encountered in sqrt
  IV = np.sqrt(svi_model(theta, k_new, ttm) / ttm)


NaN values found in IV for IV_matrix_2022-11-20.csv.
NaN values found in IV for IV_matrix_2022-11-22.csv.
NaN values found in IV for IV_matrix_2022-11-28.csv.


In [8]:
import os
import pandas as pd

r2_mapping = {
    "_098": "0.98",
    "_095": "0.95",
    "_090": "0.90",
    "_080": "0.80",
    "_070": "0.70",
    "_060": "0.60"
}

ttm_mapping = {
    "IV5": "ttm5",
    "IV9": "ttm9",
    "IV14": "ttm14",
    "IV27": "ttm27"
}

summary_table = []

for filename in os.listdir(os.path.join(path, "IR0","moneyness")):
    if filename.endswith(".csv"):
        # Extract the R2 and TTM values from the filename
        r2_value = next((value for key, value in r2_mapping.items() if key in filename), None)
        ttm_value = next((value for key, value in ttm_mapping.items() if key in filename), None)

        # Skip files that don't match
        if r2_value and ttm_value:
            # Count the number of rows in the file
            file_path = os.path.join(path, "IR0","moneyness", filename)
            num_rows = len(pd.read_csv(file_path))

            # Append the data to the summary table
            summary_table.append({
                "R2": r2_value,
                ttm_value: num_rows
            })

# Convert the summary table to a DataFrame
summary_df = pd.DataFrame(summary_table)

# Group by R2 and aggregate the counts
final_summary = summary_df.groupby("R2").sum().reset_index()

# Reset the order of the columns and the rows
final_summary = final_summary[["R2"] + list(ttm_mapping.values())]
final_summary = final_summary.sort_values(by="R2", ascending=False).reset_index(drop=True)

# save the final summary to a CSV file
final_summary.to_csv(os.path.join(path, "IR0_moneyness_summary.csv"), index=False)    